# EODHD Setup Guide

**Provider:** EODHD (End of Day Historical Data)

**Best for:** Global markets (70+ exchanges), ETFs, mutual funds, bonds, economic data

**Time:** 5 minutes

**Cost:** FREE tier (20 API calls/day), All World $20/mo, Extended $80/mo

---

**🎯 This is part of the STARTER TIER for the course!**

**Why EODHD?** Best coverage for global markets

## Step 1: Create EODHD Account

1. Go to [eodhd.com](https://eodhd.com/)
2. Sign up

## Step 2: Get Your API Key

1. Go to [eodhd.com/cp/dashboard](https://eodhd.com/cp/dashboard)
2. Your API token is shown on the page
3. **Copy the key**


## Step 3: Set Your API Key

Run this cell and paste your API key when prompted:

In [17]:
import os
from getpass import getpass

# Prompt for API key (hidden input)
api_key = getpass("Enter your EODHD API key: ")
os.environ["EODHD_API_KEY"] = api_key

print("✅ API key set!")
print(f"   Key format: {api_key[:8]}...{api_key[-4:]}")

Enter your EODHD API key: ··········
✅ API key set!
   Key format: 63ef0dcf...0144


## Step 4: Install Required Packages

In [18]:
!pip install -q requests pandas

## Step 5: Test Your Connection - Real-time Quote

In [19]:
import requests

BASE_URL = "https://eodhd.com/api"

def get_realtime_quote(ticker: str, exchange: str = "US") -> dict:
    """Get real-time quote from EODHD.

    Args:
        ticker: Stock symbol
        exchange: Exchange code (US, LSE, XETRA, etc.)

    Returns:
        Dictionary with quote data or error info
    """
    url = f"{BASE_URL}/real-time/{ticker}.{exchange}"
    params = {
        "api_token": os.environ["EODHD_API_KEY"],
        "fmt": "json"
    }

    try:
        response = requests.get(url, params=params)

        # Check for empty response (common with unsupported exchanges/tickers)
        if not response.text or response.text.strip() == "":
            return {"error": f"Empty response for {ticker}.{exchange} - may require paid tier"}

        return response.json()

    except requests.exceptions.JSONDecodeError:
        return {"error": f"Invalid response for {ticker}.{exchange}"}
    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {str(e)}"}

# Test with Apple
quote = get_realtime_quote("AAPL", "US")

if "code" in quote:
    print("✅ Connection successful!")
    print(f"   Symbol: {quote.get('code')}")
    print(f"   Price: ${quote.get('close')}")
    print(f"   Open: ${quote.get('open')}")
    print(f"   High: ${quote.get('high')}")
    print(f"   Low: ${quote.get('low')}")
    print(f"   Volume: {quote.get('volume'):,}")
    print(f"   Change: {quote.get('change_p', 0):.2f}%")
elif "error" in quote:
    print(f"❌ {quote['error']}")
else:
    print("❌ Connection failed")
    print(f"   Response: {quote}")

✅ Connection successful!
   Symbol: AAPL.US
   Price: $249.85
   Open: $249.2
   High: $250.78
   Low: $248.385
   Volume: 8,690,063
   Change: 0.89%


## Step 6: Get Historical End-of-Day Data

In [20]:
import pandas as pd

def get_historical_data(ticker: str, exchange: str = "US", period: str = "d") -> pd.DataFrame:
    """Get historical end-of-day data.

    Args:
        ticker: Stock symbol
        exchange: Exchange code
        period: 'd' (daily), 'w' (weekly), 'm' (monthly)

    Returns:
        DataFrame with historical data, or empty DataFrame on error
    """
    url = f"{BASE_URL}/eod/{ticker}.{exchange}"
    params = {
        "api_token": os.environ["EODHD_API_KEY"],
        "fmt": "json",
        "period": period,
        "order": "d"  # descending (newest first)
    }

    try:
        response = requests.get(url, params=params)

        # Check for empty response
        if not response.text or response.text.strip() == "":
            print(f"⚠️ Empty response for {ticker}.{exchange}")
            return pd.DataFrame()

        data = response.json()

        if isinstance(data, list) and len(data) > 0:
            df = pd.DataFrame(data)
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
            return df

        return pd.DataFrame()

    except Exception as e:
        print(f"⚠️ Error fetching {ticker}.{exchange}: {e}")
        return pd.DataFrame()

# Get Apple's historical data
df = get_historical_data("AAPL", "US")

if not df.empty:
    print("✅ Historical data retrieved!")
    print(f"   Period: {df.index[-1].date()} to {df.index[0].date()}")
    print(f"   Records: {len(df)}")
    print(f"\nLast 5 days:")
    print(df.head())
else:
    print("❌ Failed to retrieve data")

✅ Historical data retrieved!
   Period: 1980-12-12 to 2026-01-21
   Records: 11368

Last 5 days:
              open    high     low   close  adjusted_close    volume
date                                                                
2026-01-21  248.70  251.56  245.18  247.65          247.65  54446100
2026-01-20  252.73  254.79  243.42  246.70          246.70  80267500
2026-01-16  257.90  258.90  254.93  255.53          255.53  72142800
2026-01-15  260.65  261.04  257.05  258.21          258.21  39388600
2026-01-14  259.49  261.82  256.71  259.96          259.96  40019400


## Step 7: Get Global Market Data (LSE, XETRA)

In [21]:
# Get UK stock (London Stock Exchange)
print("🇬🇧 UK Stock (London Stock Exchange):")
uk_quote = get_realtime_quote("VOD", "LSE")  # Vodafone
if "code" in uk_quote:
    print(f"   Vodafone (VOD.LSE): £{uk_quote.get('close')}")
elif "error" in uk_quote:
    print(f"   ⚠️ {uk_quote['error']}")

# Get German stock (XETRA)
print("\n🇩🇪 German Stock (XETRA):")
de_quote = get_realtime_quote("SAP", "XETRA")  # SAP
if "code" in de_quote:
    print(f"   SAP (SAP.XETRA): €{de_quote.get('close')}")
elif "error" in de_quote:
    print(f"   ⚠️ {de_quote['error']}")

# Get Canadian stock (TSX)
print("\n🇨🇦 Canadian Stock (Toronto):")
ca_quote = get_realtime_quote("RY", "TO")  # Royal Bank of Canada
if "code" in ca_quote:
    print(f"   Royal Bank (RY.TO): C${ca_quote.get('close')}")
elif "error" in ca_quote:
    print(f"   ⚠️ {ca_quote['error']}")

🇬🇧 UK Stock (London Stock Exchange):
   Vodafone (VOD.LSE): £104.2

🇩🇪 German Stock (XETRA):
   SAP (SAP.XETRA): €189.74

🇨🇦 Canadian Stock (Toronto):
   Royal Bank (RY.TO): C$232.52


## Step 8: Get ETF and Mutual Fund Data

In [22]:
# Get ETF data
etfs = {
    "SPY.US": "S&P 500 ETF",
    "QQQ.US": "Nasdaq 100 ETF",
    "VTI.US": "Total Stock Market ETF",
    "VWO.US": "Emerging Markets ETF"
}

print("📊 ETF Prices:")
for symbol, name in etfs.items():
    ticker, exchange = symbol.split(".")
    quote = get_realtime_quote(ticker, exchange)
    if "close" in quote:
        print(f"   {name} ({symbol}): ${quote.get('close')}")
    elif "error" in quote:
        print(f"   {name} ({symbol}): ⚠️ {quote['error']}")

📊 ETF Prices:
   S&P 500 ETF (SPY.US): $687.14
   Nasdaq 100 ETF (QQQ.US): $618.05
   Total Stock Market ETF (VTI.US): $340.0799
   Emerging Markets ETF (VWO.US): $56.5401


## Step 9: Get Fundamental Data (Extended Tier)

⚠️ **Note:** Fundamentals require the higher tiers. This cell will show an error on free tier - that's expected! For free fundamental data, use yfinance.

In [24]:
def get_fundamentals(ticker: str, exchange: str = "US") -> dict:
    """Get fundamental data (requires Extended tier $80/mo).

    Args:
        ticker: Stock symbol
        exchange: Exchange code

    Returns:
        Dictionary with fundamental data, or error info
    """
    url = f"{BASE_URL}/fundamentals/{ticker}.{exchange}"
    params = {
        "api_token": os.environ["EODHD_API_KEY"],
        "fmt": "json"
    }

    try:
        response = requests.get(url, params=params)

        # Check for empty response
        if not response.text or response.text.strip() == "":
            return {"error": "empty_response", "message": "Fundamentals require Extended tier ($80/mo)"}

        # Check for non-JSON response (sometimes returns error page)
        if not response.text.strip().startswith('{') and not response.text.strip().startswith('['):
            return {"error": "invalid_response", "message": "Fundamentals require Extended tier ($80/mo)"}

        return response.json()

    except Exception as e:
        # JSONDecodeError usually means empty/invalid response
        if "Expecting value" in str(e):
            return {"error": "no_data", "message": "Fundamentals require Extended tier ($80/mo)"}
        return {"error": "exception", "message": str(e)}

# Get Apple fundamentals
fundamentals = get_fundamentals("AAPL", "US")

if "General" in fundamentals:
    general = fundamentals["General"]
    highlights = fundamentals.get("Highlights", {})

    print("✅ Fundamental data retrieved!")
    print(f"\n📊 {general.get('Name')} ({general.get('Code')})")
    print(f"   Sector: {general.get('Sector')}")
    print(f"   Industry: {general.get('Industry')}")
    print(f"   Market Cap: ${highlights.get('MarketCapitalization', 0)/1e9:.1f}B")
    print(f"   P/E Ratio: {highlights.get('PERatio', 'N/A')}")
    print(f"   EPS: ${highlights.get('EarningsShare', 'N/A')}")
    print(f"   Dividend Yield: {highlights.get('DividendYield', 0)*100:.2f}%")
elif "error" in fundamentals:
    print(f"⚠️ {fundamentals.get('message', 'Unknown error')}")
    print("   💡 Fundamentals data requires the higher tier")
    print("   📊 For free fundamental data, try yfinance")
else:
    print("❌ Failed to retrieve fundamentals")

⚠️ Fundamentals require Extended tier ($80/mo)
   💡 Fundamentals data requires the higher tier
   📊 For free fundamental data, try yfinance


## Step 10: Create Tools for smolagents

In [25]:
!pip install -q smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 4.4 MB/s eta 0:00:00


In [26]:
from smolagents import tool

@tool
def get_global_stock_price(ticker: str, exchange: str = "US") -> dict:
    """Get real-time stock price from global markets via EODHD.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'VOD', 'SAP')
        exchange: Exchange code - 'US' (default), 'LSE' (London), 'XETRA' (Germany),
                  'TSE' (Tokyo), 'HKEX' (Hong Kong), 'ASX' (Australia)

    Returns:
        Dictionary with price, change, volume from the specified exchange
    """
    import requests
    import os

    url = f"https://eodhd.com/api/real-time/{ticker}.{exchange}"
    params = {
        "api_token": os.environ["EODHD_API_KEY"],
        "fmt": "json"
    }

    try:
        response = requests.get(url, params=params)

        # Check for empty response
        if not response.text or response.text.strip() == "":
            return {"error": f"No data for {ticker}.{exchange} - may require paid tier", "ticker": ticker}

        quote = response.json()

        if "code" in quote:
            return {
                "ticker": f"{ticker}.{exchange}",
                "price": quote.get("close"),
                "open": quote.get("open"),
                "high": quote.get("high"),
                "low": quote.get("low"),
                "volume": quote.get("volume"),
                "change_percent": quote.get("change_p"),
                "exchange": exchange,
                "source": "EODHD"
            }

        return {"error": "Failed to retrieve data", "ticker": ticker}

    except Exception as e:
        return {"error": str(e), "ticker": ticker}

@tool
def get_etf_data(ticker: str) -> dict:
    """Get ETF price and basic info from EODHD.

    Args:
        ticker: ETF symbol (e.g., 'SPY', 'QQQ', 'VTI', 'VWO')

    Returns:
        Dictionary with ETF price and details
    """
    import requests
    import os

    # Real-time quote
    url = f"https://eodhd.com/api/real-time/{ticker}.US"
    params = {
        "api_token": os.environ["EODHD_API_KEY"],
        "fmt": "json"
    }

    try:
        response = requests.get(url, params=params)

        # Check for empty response
        if not response.text or response.text.strip() == "":
            return {"error": f"No data for {ticker}", "ticker": ticker}

        quote = response.json()

        # Known ETF names
        etf_names = {
            "SPY": "SPDR S&P 500 ETF Trust",
            "QQQ": "Invesco QQQ Trust",
            "VTI": "Vanguard Total Stock Market ETF",
            "VWO": "Vanguard FTSE Emerging Markets ETF",
            "IWM": "iShares Russell 2000 ETF",
            "EFA": "iShares MSCI EAFE ETF",
            "AGG": "iShares Core U.S. Aggregate Bond ETF",
            "GLD": "SPDR Gold Shares"
        }

        if "code" in quote:
            return {
                "ticker": ticker.upper(),
                "name": etf_names.get(ticker.upper(), "ETF"),
                "price": quote.get("close"),
                "change_percent": quote.get("change_p"),
                "volume": quote.get("volume"),
                "source": "EODHD"
            }

        return {"error": "Failed to retrieve data", "ticker": ticker}

    except Exception as e:
        return {"error": str(e), "ticker": ticker}

# Test the tools
print("Tool 1 - Global Stock Price:")
print(get_global_stock_price("AAPL", "US"))

print("\nTool 2 - ETF Data:")
print(get_etf_data("SPY"))

Tool 1 - Global Stock Price:
{'ticker': 'AAPL.US', 'price': 249.6577, 'open': 249.2, 'high': 250.78, 'low': 248.385, 'volume': 8748715, 'change_percent': 0.8107, 'exchange': 'US', 'source': 'EODHD'}

Tool 2 - ETF Data:
{'ticker': 'SPY', 'name': 'SPDR S&P 500 ETF Trust', 'price': 687.14, 'change_percent': 0.2539, 'volume': 17155284, 'source': 'EODHD'}


## ✅ Setup Complete!

You're ready to use EODHD in the course notebooks.

**Next steps:**
- Save your API key securely (password manager)
- Consider All World tier ($20/mo) for full global access
- Explore the 70+ supported exchanges
- Continue to the course Module 1 notebook
